In [1]:
# install Cornac and PyTorch (VBPR model implementation uses PyTorch)
!pip3 install cornac torch>=0.4.1

In [2]:
import cornac
from cornac.datasets import tradesy
from cornac.data import ImageModality
from cornac.eval_methods import RatioSplit
import pandas as pd
import numpy as np
from cornac.data import Reader
from cornac.hyperopt import Discrete, Continuous
from cornac.hyperopt import GridSearch, RandomSearch


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
reader=Reader()
feedback= reader.read("/content/drive/MyDrive/MITB/CS608/Project 2/amazon_feedback_full_1.2m.csv",sep=',',skip_lines=1)

In [5]:
feedback[:5]

[('A2MOB79WI93FDN', '6040972467', 5.0),
 ('A16O44MQ3ZS7CL', '6040972467', 3.0),
 ('A14ZLDIG0XD6AL', '6040972467', 3.0),
 ('AZD378NGJIDQH', '6040972467', 5.0),
 ('A339XOY8DU2UWE', '6040972467', 3.0)]

In [6]:
len(feedback)

1287045

In [7]:
df=pd.read_csv('/content/drive/MyDrive/MITB/CS608/Project 2/full_item_feats2.csv')

df.head()

,asin,feat
0,6040596368,"[0.1650523888, 0.524156368, 0.2667483240000000..."
1,6040972467,"[0.4050605, 0.33036426, 0.2321943, 0.15210894,..."
2,6040974753,"[0.33511356, 0.34719497, 0.115295, 0.08025931,..."
3,9789892837,"[0.052969208625, 0.41584283375000003, 0.255046..."
4,B00008L1ST,"[0.07350600350000001, 0.17075585066666668, 0.0..."


In [8]:
item_ids=df['asin'].tolist()
item_ids[:5]

['6040596368', '6040972467', '6040974753', '9789892837', 'B00008L1ST']

In [9]:
len(item_ids)

42951

In [10]:
def floatit(x):
  return [float(i) for i in x]

In [11]:
features_=df['feat'].apply(lambda x: x[1:-1].split(','))
features=np.asarray(features_.apply(floatit).tolist())


In [12]:
features[:5]

array([[0.16505239, 0.52415637, 0.26674832, ..., 0.38159088, 0.29361678,
        0.1017647 ],
       [0.4050605 , 0.33036426, 0.2321943 , ..., 0.22281997, 0.2165342 ,
        0.23516575],
       [0.33511356, 0.34719497, 0.115295  , ..., 0.1206347 , 0.33069977,
        0.11935604],
       [0.05296921, 0.41584283, 0.25504655, ..., 0.28925593, 1.24776427,
        0.06325168],
       [0.073506  , 0.17075585, 0.06438219, ..., 0.17174458, 0.94780602,
        0.15769453]])

In [13]:
len(features)

42951

In [14]:
item_image_modality = ImageModality(features=features, ids=item_ids, normalized=True)

In [15]:
# Define an evaluation method to split feedback into train and test sets
ratio_split = RatioSplit(
    data=feedback,
    test_size=0.15,
    val_size=0.15,
    rating_threshold=1,
    exclude_unknowns=True,
    verbose=True,
    item_image=item_image_modality,
)

rating_threshold = 1.0
exclude_unknowns = True


/usr/local/lib/python3.7/dist-packages/cornac/data/dataset.py:361: UserWarning: 13733 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


---
Training data:
Number of users = 435707
Number of items = 41494
Number of ratings = 887198
Max rating = 5.0
Min rating = 1.0
Global mean = 4.1


/usr/local/lib/python3.7/dist-packages/cornac/data/dataset.py:361: UserWarning: 540 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


---
Test data:
Number of users = 110810
Number of items = 25284
Number of ratings = 144227
Number of unknown users = 0
Number of unknown items = 0


/usr/local/lib/python3.7/dist-packages/cornac/data/dataset.py:361: UserWarning: 534 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


---
Validation data:
Number of users = 110960
Number of items = 25340
Number of ratings = 144208
---
Total users = 435707
Total items = 41494


In [16]:
# Instantiate evaluation measures
eval_metrics = [
  cornac.metrics.AUC,
  cornac.metrics.Recall(k=20),
  cornac.metrics.NDCG(k=20),
  cornac.metrics.NCRR(k=20),
  cornac.metrics.Recall(k=50),
  cornac.metrics.NDCG(k=50),
  cornac.metrics.NCRR(k=50),
]

recall = cornac.metrics.Recall(k=20)

In [ ]:
# Instantiate AMR
amr = cornac.models.AMR(
    k=500,
    k2=500,
    n_epochs=25,
    batch_size=256,
    learning_rate=0.001,
    lambda_w=1,
    lambda_b=0.3,
    lambda_e=0.0,
    use_gpu=True,
)

In [ ]:
# Evaluation
cornac.Experiment(eval_method=ratio_split, models=[amr], metrics=eval_metrics).run()


[AMR] Training started!



Optimization finished!

[AMR] Evaluation started!




VALIDATION:
...
    | NCRR@20 | NCRR@50 | NDCG@20 | NDCG@50 | Recall@20 | Recall@50 |  Time (s)
--- + ------- + ------- + ------- + ------- + --------- + --------- + ---------
AMR |  0.0008 |  0.0014 |  0.0017 |  0.0051 |    0.0051 |    0.0211 | 1514.0417

TEST:
...
    | NCRR@20 | NCRR@50 | NDCG@20 | NDCG@50 | Recall@20 | Recall@50 | Train (s) |  Test (s)
--- + ------- + ------- + ------- + ------- + --------- + --------- + --------- + ---------
AMR |  0.0008 |  0.0014 |  0.0017 |  0.0051 |    0.0051 |    0.0213 | 4320.6544 | 1504.2731



In [ ]:
amr.save('/content/drive/MyDrive/MITB/CS608/Project 2/saved_models/amr_k500_epoch_25')

AMR model is saved to /content/drive/MyDrive/MITB/CS608/Project 2/saved_models/amr_k500_epoch_25/AMR/2021-07-19_06-47-42-843109.pkl


'/content/drive/MyDrive/MITB/CS608/Project 2/saved_models/amr_k500_epoch_25/AMR/2021-07-19_06-47-42-843109.pkl'

In [ ]:
len(feedback)*0.1

128704.5

In [ ]:
# Define an evaluation method to split feedback into train and test sets
ratio_split = RatioSplit(
    data=feedback[:128704],
    test_size=0.15,
    val_size=0.15,
    rating_threshold=1,
    exclude_unknowns=True,
    verbose=True,
    item_image=item_image_modality,
)

rating_threshold = 1.0
exclude_unknowns = True


/usr/local/lib/python3.7/dist-packages/cornac/data/dataset.py:361: UserWarning: 11514 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)
/usr/local/lib/python3.7/dist-packages/cornac/data/dataset.py:361: UserWarning: 391 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)
/usr/local/lib/python3.7/dist-packages/cornac/data/dataset.py:361: UserWarning: 398 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


---
Training data:
Number of users = 49162
Number of items = 843
Number of ratings = 78578
Max rating = 5.0
Min rating = 1.0
Global mean = 4.3
---
Test data:
Number of users = 11105
Number of items = 517
Number of ratings = 13094
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 11138
Number of items = 527
Number of ratings = 13103
---
Total users = 49162
Total items = 843


In [ ]:
# Instantiate AMR
amr = cornac.models.AMR(
    k=100,
    k2=100,
    n_epochs=10,
    batch_size=256,
    learning_rate=0.0005,
    lambda_w=1,
    lambda_b=0.1,
    lambda_e=0.0,
    lambda_adv=0.0,
    use_gpu=True,
)


# Wrap model inside GridSearch along with the searching space
amr_search = GridSearch(
    model=amr,
    space=[
        Discrete("k", [500]),
        Discrete("k2", [400]),
        Discrete("n_epochs", [10]),
        Discrete("batch_size", [256]),
        Discrete("learning_rate", [0.0005]),
        Discrete("lambda_adv", [0.3, 0.1, 0.03, 0.01, 0.001, 0.0001,0]),
        # Discrete("lambda_adv", np.linspace(0, 1, 5)),
        # Discrete("k", np.linspace(10, 100, 5)),
        # Discrete("learning_rate", np.linspace(0.001,0.002, 2)),
        # Discrete("lambda_b", np.linspace(0.1, 0.5, 3)),
        # Discrete("lambda_e", np.linspace(0.0, 0.05, 2)),
    ],
    metric=recall,
    eval_method=ratio_split,
)

In [ ]:
# Evaluation
cornac.Experiment(eval_method=ratio_split, models=[amr_search], metrics=eval_metrics).run()


[GridSearch_AMR] Training started!
Evaluating: {'batch_size': 256, 'k': 500, 'k2': 400, 'lambda_adv': 0, 'learning_rate': 0.0005, 'n_epochs': 10}



Optimization finished!
Evaluating: {'batch_size': 256, 'k': 500, 'k2': 400, 'lambda_adv': 0.0001, 'learning_rate': 0.0005, 'n_epochs': 10}



Optimization finished!
Evaluating: {'batch_size': 256, 'k': 500, 'k2': 400, 'lambda_adv': 0.001, 'learning_rate': 0.0005, 'n_epochs': 10}



Optimization finished!
Evaluating: {'batch_size': 256, 'k': 500, 'k2': 400, 'lambda_adv': 0.01, 'learning_rate': 0.0005, 'n_epochs': 10}



Optimization finished!
Evaluating: {'batch_size': 256, 'k': 500, 'k2': 400, 'lambda_adv': 0.03, 'learning_rate': 0.0005, 'n_epochs': 10}



Optimization finished!
Evaluating: {'batch_size': 256, 'k': 500, 'k2': 400, 'lambda_adv': 0.1, 'learning_rate': 0.0005, 'n_epochs': 10}



Optimization finished!
Evaluating: {'batch_size': 256, 'k': 500, 'k2': 400, 'lambda_adv': 0.3, 'learning_rate': 0.0005, 'n_epochs': 10}



Optimization finished!
Best parameter settings: {'batch_size': 256, 'k': 500, 'k2': 400, 'lambda_adv': 0, 'learning_rate': 0.0005, 'n_epochs': 10}
Recall@20 = 0.6774

[GridSearch_AMR] Evaluation started!




VALIDATION:
...
               | NCRR@20 | NCRR@50 | NDCG@20 | NDCG@50 | Recall@20 | Recall@50 | Time (s)
-------------- + ------- + ------- + ------- + ------- + --------- + --------- + --------
GridSearch_AMR |  0.1804 |  0.1859 |  0.2933 |  0.3264 |    0.6774 |    0.8380 |  10.4419

TEST:
...
               | NCRR@20 | NCRR@50 | NDCG@20 | NDCG@50 | Recall@20 | Recall@50 | Train (s) | Test (s)
-------------- + ------- + ------- + ------- + ------- + --------- + --------- + --------- + --------
GridSearch_AMR |  0.1822 |  0.1879 |  0.2935 |  0.3274 |    0.6720 |    0.8358 |  341.6826 |  10.1222



In [ ]:
# Wrap model inside GridSearch along with the searching space
amr_search = GridSearch(
    model=amr,
    space=[
        Discrete("k", [500]),
        Discrete("k2", [500]),
        Discrete("n_epochs", [5]),
        Discrete("batch_size", [256]),
        Discrete("learning_rate", [0.0005, 0.001,0.005]),
        # Discrete("k", np.linspace(10, 100, 5)),
        # Discrete("learning_rate", np.linspace(0.001,0.002, 2)),
        Discrete("lambda_b", np.linspace(0.05, 0.1, 2)),
        # Discrete("lambda_e", np.linspace(0.0, 0.05, 2)),
    ],
    metric=recall,
    eval_method=ratio_split,
)

In [ ]:
# Evaluation
cornac.Experiment(eval_method=ratio_split, models=[amr_search], metrics=eval_metrics).run()

In [ ]:
# Define an evaluation method to split feedback into train and test sets
ratio_split = RatioSplit(
    data=feedback,
    test_size=0.15,
    val_size=0.15,
    rating_threshold=1,
    exclude_unknowns=True,
    verbose=True,
    item_image=item_image_modality,
)

rating_threshold = 1.0
exclude_unknowns = True


/usr/local/lib/python3.7/dist-packages/cornac/data/dataset.py:361: UserWarning: 13539 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


---
Training data:
Number of users = 435643
Number of items = 41545
Number of ratings = 887392
Max rating = 5.0
Min rating = 1.0
Global mean = 4.1


/usr/local/lib/python3.7/dist-packages/cornac/data/dataset.py:361: UserWarning: 570 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


---
Test data:
Number of users = 110901
Number of items = 25294
Number of ratings = 144210
Number of unknown users = 0
Number of unknown items = 0


/usr/local/lib/python3.7/dist-packages/cornac/data/dataset.py:361: UserWarning: 567 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


---
Validation data:
Number of users = 110865
Number of items = 25369
Number of ratings = 144071
---
Total users = 435643
Total items = 41545


In [ ]:
# Instantiate AMR
amr = cornac.models.AMR(
    k=500,
    k2=400,
    n_epochs=10,
    batch_size=256,
    learning_rate=0.0005,
    lambda_w=1,
    lambda_b=0.1,
    lambda_e=0.0,
    lambda_adv=0.0,
    use_gpu=True,
)

In [ ]:
# Evaluation
cornac.Experiment(eval_method=ratio_split, models=[amr], metrics=eval_metrics).run()


[AMR] Training started!



Optimization finished!

[AMR] Evaluation started!




VALIDATION:
...
    | NCRR@20 | NCRR@50 | NDCG@20 | NDCG@50 | Recall@20 | Recall@50 |  Time (s)
--- + ------- + ------- + ------- + ------- + --------- + --------- + ---------
AMR |  0.0025 |  0.0031 |  0.0050 |  0.0090 |    0.0140 |    0.0338 | 1397.3561

TEST:
...
    | NCRR@20 | NCRR@50 | NDCG@20 | NDCG@50 | Recall@20 | Recall@50 | Train (s) |  Test (s)
--- + ------- + ------- + ------- + ------- + --------- + --------- + --------- + ---------
AMR |  0.0027 |  0.0034 |  0.0054 |  0.0093 |    0.0148 |    0.0339 | 2418.3553 | 1409.9821



In [ ]:
amr.save('/content/drive/MyDrive/MITB/CS608/Project 2/saved_models/amr_k500_k2_400_final')

AMR model is saved to /content/drive/MyDrive/MITB/CS608/Project 2/saved_models/amr_k500_k2_400_final/AMR/2021-07-19_10-50-00-803818.pkl


'/content/drive/MyDrive/MITB/CS608/Project 2/saved_models/amr_k500_k2_400_final/AMR/2021-07-19_10-50-00-803818.pkl'

In [17]:
# Instantiate AMR
amr = cornac.models.AMR(
    k=1000,
    k2=400,
    n_epochs=15,
    batch_size=256,
    learning_rate=0.0005,
    lambda_w=1,
    lambda_b=0.1,
    lambda_e=0.0,
    lambda_adv=0.0,
    use_gpu=True,
)

In [18]:
# Evaluation
cornac.Experiment(eval_method=ratio_split, models=[amr], metrics=eval_metrics).run()


[AMR] Training started!



Optimization finished!

[AMR] Evaluation started!




VALIDATION:
...
    | NCRR@20 | NCRR@50 | NDCG@20 | NDCG@50 | Recall@20 | Recall@50 |  Time (s)
--- + ------- + ------- + ------- + ------- + --------- + --------- + ---------
AMR |  0.0030 |  0.0038 |  0.0060 |  0.0109 |    0.0167 |    0.0404 | 2078.1412

TEST:
...
    | NCRR@20 | NCRR@50 | NDCG@20 | NDCG@50 | Recall@20 | Recall@50 | Train (s) |  Test (s)
--- + ------- + ------- + ------- + ------- + --------- + --------- + --------- + ---------
AMR |  0.0032 |  0.0040 |  0.0063 |  0.0112 |    0.0174 |    0.0411 | 6357.7829 | 2060.4245



In [19]:
train_set = amr.train_set
uid2raw = {v:k for k,v in train_set.uid_map.items()}
iid2raw = {v:k for k,v in train_set.iid_map.items()}

item_id2idx = amr.train_set.iid_map
item_idx2id = list(amr.train_set.item_ids)
img_features = amr.train_set.item_image.features

In [20]:
from collections import defaultdict

In [25]:
UID = 'A2522ZTCKCJZQT'
UIDX = train_set.uid_map[UID]
TOPK = 20

recommendations, scores = amr.rank(UIDX)
print(f"\nTOP {TOPK} RECOMMENDATIONS for {UID}")

rec_df = defaultdict(list)
for i in recommendations[:TOPK]:
  rec_df["Recommendation"].append(item_idx2id[i])
  rec_df["Score"].append(scores[i])
pd.DataFrame(rec_df)


TOP 20 RECOMMENDATIONS for A2522ZTCKCJZQT


,Recommendation,Score
0,B00U836PY4,14.885754
1,B00CHNMU1U,13.423722
2,B011KDTJ5M,12.541751
3,B01BUYP770,11.929579
4,B0030EWG5A,11.745747
5,B000N0AKYS,11.108165
6,B01AZWWG72,11.028913
7,B019PLJNPW,10.892782
8,B00N7EI7SI,10.814615
9,B01B00D79Y,10.703573


In [26]:
UID = 'A2PACJHFKI74Y3'
UIDX = train_set.uid_map[UID]
TOPK = 20

recommendations, scores = amr.rank(UIDX)
print(f"\nTOP {TOPK} RECOMMENDATIONS for {UID}")

rec_df = defaultdict(list)
for i in recommendations[:TOPK]:
  rec_df["Recommendation"].append(item_idx2id[i])
  rec_df["Score"].append(scores[i])
pd.DataFrame(rec_df)


TOP 20 RECOMMENDATIONS for A2PACJHFKI74Y3


,Recommendation,Score
0,B00J3ACTK2,9.834010
1,B013IFWWLI,9.743040
2,B00M3IZ14C,9.235919
3,B015T2GCYG,8.623542
4,B010CH2B2K,8.518440
5,B017D6PYCC,8.501100
6,B00VAPW8YA,8.167350
7,B00ZIXVK2Q,7.961284
8,B00TAESYL4,7.724346
9,B00Q0BJISS,7.667960


In [27]:
UID = 'A2OSLOUKH5VWGL'
UIDX = train_set.uid_map[UID]
TOPK = 20

recommendations, scores = amr.rank(UIDX)
print(f"\nTOP {TOPK} RECOMMENDATIONS for {UID}")

rec_df = defaultdict(list)
for i in recommendations[:TOPK]:
  rec_df["Recommendation"].append(item_idx2id[i])
  rec_df["Score"].append(scores[i])
pd.DataFrame(rec_df)


TOP 20 RECOMMENDATIONS for A2OSLOUKH5VWGL


,Recommendation,Score
0,B00DNNS81Y,22.270397
1,B012P8XQUU,21.849895
2,B00I1AL3CU,21.336081
3,B00MC481V2,20.311707
4,B00C7VHX8W,20.081665
5,B0159B00WW,19.979481
6,B006Z8H1DK,19.964958
7,B000YPKLNW,19.790419
8,B00PHTJ3B6,19.742874
9,B00638M6IC,19.714563


In [28]:
UID = 'A1HV6KKVTGMCGE'
UIDX = train_set.uid_map[UID]
TOPK = 20

recommendations, scores = amr.rank(UIDX)
print(f"\nTOP {TOPK} RECOMMENDATIONS for {UID}")

rec_df = defaultdict(list)
for i in recommendations[:TOPK]:
  rec_df["Recommendation"].append(item_idx2id[i])
  rec_df["Score"].append(scores[i])
pd.DataFrame(rec_df)


TOP 20 RECOMMENDATIONS for A1HV6KKVTGMCGE


,Recommendation,Score
0,B01AL4S4KM,4.963665
1,B00YAP29D2,4.954957
2,B01BWUJRYQ,4.666190
3,B019M88Z52,4.494749
4,B0178QOWZC,4.462755
5,B001045ZZK,4.334345
6,B01HEUA0AS,4.223036
7,B00UHYC9OO,4.177280
8,B00HV34OW4,3.947719
9,B01BUJEML2,3.919526
